## Yinghong Xu
Scraped Amazon best-seller pages in Python to create a new dataset for word tokenization in product names. Generated the high frequency words and analyzed product ranking factors. Visualized the output and created a word cloud to identify current popular products and provide sentiment analysis. A selection bar provided for the user to easily choose a category to find more information in the products.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# from __future__ import print_function
# from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from wordcloud import WordCloud

In [4]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import pandas as pd
import numpy as np

In [5]:
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize

In [6]:
lst=['Sports & Outdoors','Toys & Games','Cell Phones & Accessories', 'Health & Household', 'Baby', 'Beauty & Personal Care',\
    'Clothing, Shoes & Jewelry','Entertainment Collectibles', 'Musical Instruments', 'Pet Supplies', 'Office Products',\
    'CDs & Vinyl', 'Tools & Home Improvement', 'Industrial & Scientific', 'Arts, Crafts & Sewing', 'Electronics', \
    'Kitchen & Dining', 'Sports Collectibles']

drop_down=widgets.Dropdown(options=lst,
                value='Sports & Outdoors',
                description ='Category: ', disabled=False)


def word_token(x):
    products={}
    urls=[]
    print("")
    
    if x=='Toys & Games':
        url='https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Sports & Outdoors':
        url='https://www.amazon.com/Best-Sellers-Sports-Outdoors/zgbs/sporting-goods/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Cell Phones & Accessories':
        url='https://www.amazon.com/Best-Sellers/zgbs/wireless/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Health & Household':
        url='https://www.amazon.com/Best-Sellers-Health-Personal-Care/zgbs/hpc/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Baby':
        url='https://www.amazon.com/Best-Sellers-Baby/zgbs/baby-products/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Beauty & Personal Care':
        url='https://www.amazon.com/Best-Sellers-Beauty/zgbs/beauty/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Clothing, Shoes & Jewelry':
        url='https://www.amazon.com/Best-Sellers/zgbs/fashion/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Entertainment Collectibles':
        url='https://www.amazon.com/Best-Sellers-Entertainment-Collectibles/zgbs/entertainment-collectibles/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Musical Instruments':
        url='https://www.amazon.com/Best-Sellers-Musical-Instruments/zgbs/musical-instruments/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Pet Supplies':
        url='https://www.amazon.com/Best-Sellers-Pet-Supplies/zgbs/pet-supplies/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Office Products':
        url='https://www.amazon.com/Best-Sellers-Office-Products/zgbs/office-products/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='CDs & Vinyl':
        url='https://www.amazon.com/best-sellers-music-albums/zgbs/music/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Tools & Home Improvement':
        url='https://www.amazon.com/Best-Sellers-Home-Improvement/zgbs/hi/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Industrial & Scientific':
        url='https://www.amazon.com/Best-Sellers-Industrial-Scientific/zgbs/industrial/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Video Games':
        url='https://www.amazon.com/best-sellers-video-games/zgbs/videogames/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Arts, Crafts & Sewing':
        url='https://www.amazon.com/Best-Sellers-Arts-Crafts-Sewing/zgbs/arts-crafts/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Electronics':
        url='https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Kitchen & Dining':
        url='https://www.amazon.com/Best-Sellers-Kitchen-Dining/zgbs/kitchen/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    elif x=='Sports Collectibles':
        url='https://www.amazon.com/Best-Sellers-Sports-Collectibles/zgbs/sports-collectibles/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}'
    prd_no=0
    for page in range(3):
        urls.append(url.format(page, page))
        page=urls[1:]
   
    for p in page: 
        uClient=uReq(p)
        page_html=uClient.read()
        uClient.close()
        page_soup=soup(page_html, "html.parser")
        containers= page_soup.findAll("div",{"class":"a-section a-spacing-none aok-relative"})
         
        for container in containers:    
            if container.findAll("div", {"class": "a-section a-spacing-small"}): 
                product_name=container.findAll("div", {"class": "a-section a-spacing-small"})[0].img["alt"]
            else:
                product_name=""
                
            if container.findAll("a", {"class":"a-link-normal"}):        
                link_container=container.findAll("a", {"class":"a-link-normal"})[0]["href"]
                preflix="https://www.amazon.com"
                link=preflix+link_container
            else:
                link=""
             
            if container.findAll("span", {"class":"p13n-sc-price"}):
                price=container.findAll("span", {"class":"p13n-sc-price"})[0].text
            else:
                price=""  
        #rating
            if container.findAll("span", {"class":"a-icon-alt"}):
                rating=container.findAll("span", {"class":"a-icon-alt"})[0].text
            else:
                rating=""
        #reviews
            if container.findAll("a", {"class":"a-size-small a-link-normal"}):
                reviews=container.findAll("a", {"class":"a-size-small a-link-normal"})[0].text
            else:
                reviews=""    
        #rank
            rank=container.findAll("span", {"class":"zg-badge-text"})[0].text

            prd_no+=1
            products[prd_no]=[product_name, rank, link, price, rating, reviews]
    products_df=pd.DataFrame.from_dict(products, orient='index', \
                                       columns=['product_name', 'rank', 'link', 'price', 'rating', 'reviews'])
    
    products_df.to_csv('prds.csv', index=False)
    df=pd.read_csv('prds.csv')
    # Data cleaning
    df['rank']=df['rank'].str.strip('#')
    df.price=df.price.str.strip('$')

    tdf=df.rating.str.split(' ', expand=True)[0]
    df.rating=tdf
    df1=df
    df1 = df1[df1['rank'].notna()]
# word tokenization
    punctuation = ['(', ')', '?', ':', ',', '.', '!', '/', '"','–', '-', "'",'_','&', '[', ']', '%', '|', '#',';']
    a=df1['product_name'].str.lower().str.cat(sep=' ')
    for i in punctuation:
        a=a.replace(i, '')
    words=nltk.tokenize.word_tokenize(a)
    word_dist=nltk.FreqDist(words)
    stopwords=nltk.corpus.stopwords.words('english')
    words_dist_new=nltk.FreqDist(w for w in words if w not in stopwords)

    rslt=pd.DataFrame(words_dist_new.most_common(len(words_dist_new)), columns=['VocaList', 'Count']).set_index('VocaList')  
    # plot

    rslt.head(20).plot(kind='bar', figsize=(10,5))
    plt.title("Top 20 Frequency Words", fontsize=20)
    plt.xlabel('VocaList', fontsize=15)
    plt.ylabel("Count", fontsize=15)
    plt.xticks(rotation=60, fontsize=15)

    # word cloud
    wordcloudConvertDF = rslt.T.to_dict('records')
    wordcloud = WordCloud(width=800, height=600, random_state=21, max_font_size=100, relative_scaling=0.6, colormap='Dark2')\
        .generate_from_frequencies(dict(*wordcloudConvertDF))
    plt.figure(figsize=(12, 12))    
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title("Word Cloud", fontsize=20)
    plt.axis('off')
    plt.show()
    
    # ranking factors
    df_new=df1.dropna(axis='rows')
    try:
        df_new['reviews']=df_new['reviews'].str.replace(',','')
        df_new['reviews'] = df_new['reviews'].astype(int)
    except:
        pass
    df_new['price'] = pd.to_numeric(df_new['price'])
    df_new['rank'] = pd.to_numeric(df_new['rank'])
    df_new['rating'] = pd.to_numeric(df_new['rating'])
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4))
    fig.suptitle('Ranking Factors', size=20, y=1.02)

    df_new.plot(x='price', y='rank', kind='scatter', color='orange', ax=ax2)
    df_new.plot(x='reviews', y='rank', kind='scatter', color='green', ax=ax1)
    df_new.plot(x='rating',y='rank', kind='scatter', color='black', ax=ax3)
    return



In [7]:
widgets.interact(word_token, x=drop_down)

interactive(children=(Dropdown(description='Category: ', options=('Sports & Outdoors', 'Toys & Games', 'Cell P…

<function __main__.word_token(x)>